# Willkommen!





PROJECT FOR PDF SCRAPPING 
You can learn more about Google Colab here:
https://colab.research.google.com/notebooks/intro.ipynb

**Goal: Scrape unstructured legal text from public database and convert into xml readible format for further legal analysis**



In [1]:
import os
import re
import pdfplumber
import pandas as pd
import collections
from collections import namedtuple

# CODE

In [2]:
# Load the data 
from src.load import load_pdf

In [3]:

df_raw = load_pdf() # this should take a while to extract all data

In [4]:
df_raw.head(2)

,text,page
0,\n \nEBA/GL/2021/02 \n1 March 2021 \nFinal Re...,0
1,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,1


In [5]:
# get a sample 
df_raw_sample = df_raw[:70]

# get a text from a page to run test against the regex
df_raw_text_sample = df_raw["text"].iloc[54]
df_raw_text_sample


'FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DILIGENCE AND THE FACTORS CREDIT  \nAND FINANCIAL INSTITUTIONS SHOULD CONSIDER WHEN ASSESSING THE ML/TF RISK  \nASSOCIATED WITH INDIVIDUAL BUSINESS RELATIONSHIPS AND OCCASIONAL TRANSACTIONS \n4.68.  Firms should note that the application of a risk-based approach does not of itself require \nthem to refuse, or terminate, business relationships with entire categories of customers that \nthey associate with higher ML/TF risk, as the risk associated with individual business \nrelationships will vary, even within one category. \nMonitoring  \n4.69.  Pursuant to Article 13 of Directive (EU) 2015/849, firms should monitor their business \nrelationships with their customers. \n4.70.  Monitoring should include: \na.  Monitoring of transactions to ensure that these are in line with the customer’s \nrisk profile, their financial situation, and the firm’s wider knowledge of the \ncustomer to detect unusual or suspicious transactions; and  \nb.  keeping t

**2. Build Regex**

NOTE: These need fixing for subletters

In [28]:
# TEST REGEX ON PDF 
# Let's first replace the line breaks as the regex below can't handle line breaks. It's just easier to remove them first.
text_cleaned = df_raw_text_sample

# I Sections 

Section_re = r'(Guideline) (\d+\:) (.*)'
Sections = re.findall(Section_re, text_cleaned, re.IGNORECASE)
# print(matches_group0)


# 1. Paragraphs

Paragraph_re = r'(\d+\.\d+\.)'
Paragraphs = re.findall(Paragraph_re, text_cleaned, re.IGNORECASE)
# print(matches_group1)

# ## Text after paragraphs
Paragraph_text_re = r'^\d+(?:\.\d+)+\. (.*(?:\r?\n(?!\d+\.).*)*)'
Paragraph_text = re.findall(Paragraph_text_re, text_cleaned, re.MULTILINE)
print(len(Paragraph_text))


# a) Test for subletters specifically

subletters_re = r'^(?:(?:\d+(?:\.)+|[a-zA-Z]|[a-zA-Z]{2}|[a-zA-Z]{3})[.)])'
Subletters = re.findall(subletters_re, text_cleaned, re.MULTILINE)
#print(Subletters)

# a) text: Text after sub-letters 
subletters_text_re = r'^(?:(?:\d+(?:\.)+|[a-zA-Z]|[a-zA-Z]{2}|[a-zA-Z]{3})[.)]) (.*(?:\r?\n(?![a-zA-Z\d]+[.)]).*)*)'
Subletters_text = re.findall(subletters_text_re , text_cleaned, re.MULTILINE)
# #print(Subletters_text)

# # aa) Sub-sub letters
# subsubletters_re = r'^[a-zA-Z\s]{2}[\.)]{1}'
# subsubletters = re.findall(subsubletters_re, text_cleaned, re.MULTILINE)
# #print(subsubletters)

# # # aa): Text after sub-sub-letters 
# # subsubletters_text_re = r'[a-z][a-z]\)'
# # Subsubletters_text = re.findall(subsubletters_text_re, text_cleaned, re.MULTILINE)
# # #print(Subsubletters_text)


# # # a. Sub-letters 3rd degree
# sub_point_re = r'^[a-zA-Z\s]{3}[\.)]{1}'
# Subsubletters_point = re.findall(sub_point_re, text_cleaned, re.MULTILINE)
# #print(Subsubletters_point)

# # a.  Sub-letter text with point
# sub_point_text_re = r'([A-z]\.M{0,2}).*'
# sub_point_text = re.findall(sub_point_text_re , text_cleaned, re.MULTILINE)
# #print(sub_point_text)



# # i.ii.iv Test for sub-strings

# sub_letter_re = r'(?=\b[MDCLXVI]+\b)M{0,4}(?:CM|CD|D?C{0,3})(?:XC|XL|L?X{0,3})(?:IX|IV|V?I{0,3})'
# Substrings = re.findall(sub_letter_re, text_cleaned,  re.IGNORECASE)
# # print(matches_group5)

# # Test 4.1:  sub-strings text
# sub_letter_text_re = r'(?=\b[MDCLXVI]+\b)M{0,4}(?:CM|CD|D?C{0,3})(?:XC|XL|L?X{0,3})(?:IX|IV|V?I{0,3})(.*)'

# Substrings_text = re.findall(sub_letter_text_re, text_cleaned ,  re.IGNORECASE)
# # print(matches_group6)


# PRINT

# print(f"Original text (whitout line breaks):{text_cleaned }")
# print("----------------------------------------")
print(f"Extracted text for Guidelines: {Paragraphs }")
print(f"Extracted text for Guideline text: {Paragraph_text }")
# # print("----------------------------------------")
# #--------------------
print(f"Extracted Subletter: {Subletters}")
# # print(f"Extracted Subletters text: {Subletters_text}")
# print("----------------------------------------")
# #--------------------
# print(f"Extracted Sublsubetter: {subsubletters}")
# print(f"Extracted Subsubletters text: {Subsubletters_text}")
# print("----------------------------------------")
#print(f"Extracted Substrings: {Subsubletters_point}")
# print(f"Extracted Substrings text: {Substrings_text}")
# print("----------------------------------------")
# #--------------------
#print(f"Extracted  Sub-letter with point : {Subsubletters_point}")
# print(f"Extracted  Sub-letter text with point : {sub_point_text}")


# NEXT STEPS
# transform expression 
# def my_function(text):
        # return "something"




7
Extracted text for Guidelines: ['4.68.', '4.69.', '4.70.', '4.71.', '4.72.', '4.73.', '4.74.']
Extracted text for Guideline text: [' Firms should note that the application of a risk-based approach does not of itself require \nthem to refuse, or terminate, business relationships with entire categories of customers that \nthey associate with higher ML/TF risk, as the risk associated with individual business \nrelationships will vary, even within one category. \nMonitoring  ', ' Pursuant to Article 13 of Directive (EU) 2015/849, firms should monitor their business \nrelationships with their customers. ', ' Monitoring should include: \na.  Monitoring of transactions to ensure that these are in line with the customer’s \nrisk profile, their financial situation, and the firm’s wider knowledge of the \ncustomer to detect unusual or suspicious transactions; and  \nb.  keeping the documents, data or information they hold up to date, with a view \nto understanding whether the risk associated w

# Extract Paragraphs

In [55]:
def get_paragraphs(data):
    
    para_extract = [ ]
    for row in data.iterrows():
        
        text = row[1]["text"]
        page = row[1]["page"]
       
        # Regex
        Paragraph_re = r'(\d+\.\d+\.)'
        Paragraphs = re.findall(Paragraph_re, text, re.IGNORECASE)
        
        if Paragraphs:
            for index, para in enumerate(Paragraphs):
       
                item = {}
                if (index + 1 < len(Paragraphs)):
                    ind1 = text.index(para)
                    ind2 = text.index(Paragraphs[index + 1])
                    res = text[ind1 + len(para) : ind2]
        
                    item["page"] = page
                    item["paragraph"] = para
                    item["paragraph_text"] = res
        
                    para_extract.append(item)
            
    return pd.DataFrame(para_extract)
 
paragraphs_df = get_paragraphs(df_raw)
paragraphs_df.head(4)

,page,paragraph,paragraph_text
0,8,1.9.,b) ii) c) more consistent with the principles...
1,8,1.12.,\nNon-face-to-face interactions \n15. Many r...
2,10,17.13.,\nEditorial amendments \n24. Many of the com...
3,19,1.1.,Firms should ensure that they have a thoroug...


# Combine new dataframe(paragraphs_df) with original df

In [8]:
# Combine parapgrafs with original df

df2 = pd.merge(df_raw_sample, paragraphs_df, how="left", on="page")

# Fill NaN values
df2 = df2.fillna("N/A")
df2[110:128]

,text,page,paragraph,paragraph_text
110,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,50,4.56.,When performing CDD measures or during the c...
111,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,50,4.57.,"Notwithstanding guidelines 4.54 and 4.56, fi..."
112,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,50,4.58.,To comply with Article 19 of Directive (EU) ...
113,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,50,4.59.,Firms should refer to Title II for guideline...
114,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,51,4.61.,These EDD measures should enable the firm to...
115,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,51,4.62.,"In all other high risk situations, firms sho..."
116,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,51,4.63.,Firms are not required to apply all the EDD ...
117,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,52,N/A,N/A
118,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,53,4.65.,Title II lists additional EDD measures that ...
119,FINAL REPORT ON GUIDELINES ON CUSTOMER DUE DIL...,53,4.66.,Firms should not enter into a business relat...


# Extract Subletters

The following iterates over the list of Subletters and extracts the matched subletters plus the text after that

In [41]:
def get_subletters(data):
    
    sub_extract = [ ]
    
    for row in data.iterrows():
        
        text = row[1]["paragraph_text"]
        page = row[1]["page"]
        
        # only get subletters for text, skip missing values (N/A)
        if text != "N/A":
     
            # Regex

            subletters_re = r'^(?:(?:\d+(?:\.)+|[a-zA-Z]|[a-zA-Z]{2}|[a-zA-Z]{3})[.)])'
            Subletters = re.findall(subletters_re, text, re.MULTILINE)
    
            if Subletters:
    
                for index, para in enumerate(Subletters):

                    item = {}
                    if (index + 1 < len(Subletters)):
                        ind1 = text.index(para)
                        ind2 = text.index(Subletters[index + 1])
                        res = text[ind1 + len(para) : ind2]
        
                        item["page"] = page
                        item["subletter"] = para
                        item["subletter_text"] = res
        
                        sub_extract.append(item)
            
            
    return pd.DataFrame(sub_extract)

subletters_df = get_subletters(df2)
subletters_df.head(5)


,page,subletter,subletter_text
0,54,a.,Monitoring of transactions to ensure that th...


# Combine new dataframe(paragraphs_df) with original df

In [53]:
# Combine parapgrafs with original df

df3 = pd.merge(df2, subletters_df, how="left", on="page")
df3.head(3)

cols = list(df3.columns.values) # to reorder the columns
# cols

df_final = df3[['page', 'paragraph', 'subletter', 'paragraph_text', 'subletter_text']]
df_final

,page,paragraph,subletter,paragraph_text,subletter_text
0,0,N/A,NaN,N/A,NaN
1,1,N/A,NaN,N/A,NaN
2,2,N/A,NaN,N/A,NaN
3,3,N/A,NaN,N/A,NaN
4,4,N/A,NaN,N/A,NaN
...,...,...,...,...,...
157,67,9.2.,NaN,Owing to the nature of the products and serv...,NaN
158,67,9.3.,NaN,Banks should consider the following risk fac...,NaN
159,67,9.4.,NaN,The following factors may contribute to incr...,NaN
160,68,N/A,NaN,N/A,NaN


# Write output to csv file

In [54]:
df_final.to_csv("output.csv")